In [ ]:
# Install TF object detection API
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython tf_slim
!git clone https://github.com/tensorflow/models.git

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

import os
os.environ['PYTHONPATH'] += ":/content/models"

import sys
sys.path.append("/content/models")
!python object_detection/builders/model_builder_test.py

In [ ]:
# Connect Google Drive in order to save data (if reload colab session, all data would be lost)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Set tensorflow version to 1.15 
%tensorflow_version 1.x

In [ ]:
# Creating TensorFlow frozen graph with compatible ops that can be used with TensorFlow lite
!python object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path='/content/drive/MyDrive/bagchair/models/ssd_mobilenet_v2_coco_2018_03_29/pipeline.config' \
    --trained_checkpoint_prefix='/content/drive/MyDrive/bagchair/out/model.ckpt-68252' \
    --output_directory='/content/drive/MyDrive/bagchair/frozen_tflite' \
    --add_postprocessing_op=true

In [ ]:
# Create .tflite file
!tflite_convert \
    --graph_def_file='/content/drive/MyDrive/bagchair/frozen_tflite/tflite_graph.pb' \
    --output_file='/content/drive/MyDrive/bagchair/frozen_tflite/detect.tflite' \
    --output_format=TFLITE \
    --input_shapes=1,300,300,3 \
    --input_arrays=normalized_input_image_tensor \
    --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
    --inference_type=FLOAT \
    --mean_values=128 \
    --std_dev_values=127 \
    --change_concat_input_ranges=false \
    --allow_custom_ops